# Créer un filtre anti-spam avec Naive Bayes

Dans ce projet, nous allons créer un filtre anti-spam pour les messages SMS en utilisant < multinomial Naive Bayes algorithm >. Notre objectif est d'écrire un programme qui classe les nouveaux messages avec une précision supérieure à 80% - nous nous attendons donc à ce que plus de 80% des nouveaux messages soient correctement classés comme spam ou ham (non-spam).

Pour entraîner l'algorithme, nous utiliserons un ensemble de données de 5 572 SMS déjà classés par les humains. L'ensemble de données a été constitué par Tiago A. Almeida et José María Gómez Hidalgo, et il peut être téléchargé à partir du [référentiel UCI Machine Learning](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). Le processus de collecte des données est décrit plus en détail sur cette [page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), où vous pouvez également trouver certains des articles rédigés par Tiago A. Almeida et José María Gómez Hidalgo.

## Explorer l'ensemble de données

In [1]:
import pandas as pd
sms_spam = pd.read_csv('OneDrive\Documents\my_datasets\SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
sms_spam.shape

(5572, 2)

In [2]:
sms_spam.head(3)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [3]:
sms_spam['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

nous voyons qu'environ 87% des messages sont des ham, et les 13% restants sont du spam. Cet échantillon semble représentatif, car dans la pratique, la plupart des messages que les gens reçoivent sont du ham.

Nous allons maintenant diviser notre ensemble de données en un ensemble d'entraînement et un ensemble de tests, où l'ensemble d'entraînement représente 80% des données et l'ensemble de test les 20% restants.

In [4]:
data_randomized = sms_spam.sample(frac=1, random_state=1)

training_test_index = round(len(data_randomized) * 0.8)

training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [5]:
test_set['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

In [6]:
training_set['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

comme nous nous y attendons, les pourcentages sont proches de ce que nous avons dans l'ensemble de données complet, où environ 87% des messages sont du ham, et les 13% restants sont du spam.

## Nettoyage des données

Pour calculer toutes les probabilités requises par l'algorithme, nous devrons d'abord effectuer un peu de nettoyage des données pour amener les données dans un format qui nous permettra d'extraire facilement toutes les informations dont nous avons besoin.

Essentiellement, nous voulons amener les données à ce format:

![image](https://www.zupimages.net/up/20/41/ne2l.png)

### Lettres majuscules et ponctuation

Nous allons commencer par supprimer toute la ponctuation et mettre chaque lettre en minuscule.

In [7]:
import re
training_set['SMS'] = training_set['SMS'].apply(lambda x: re.sub('\W', ' ', x))
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head(5)

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


## Création du vocabulary

Passons maintenant à la création du vocabulary, qui dans ce contexte signifie une liste avec tous les mots uniques de notre ensemble de formation.

In [8]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []

for row in training_set['SMS']:
    for word in row:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

len(vocabulary)

7783

Il semble qu'il y ait 7 783 mots uniques dans tous les messages de notre ensemble de formation.

### training_set final

Nous allons maintenant utiliser le vocabulaire que nous venons de créer pour effectuer la transformation de données souhaitée.

In [9]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

In [10]:
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [11]:
words = pd.DataFrame(word_counts_per_sms)

In [12]:
words.head(5)

,randy,prob,lived,responce,dha,garbage,pm,sem,village,certificate,...,workage,grow,acl03530150pm,woohoo,drunken,pose,nimya,dancin,26th,checking
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
training_set_clean = pd.concat([training_set, words], axis=1)

In [14]:
training_set_clean.head(5)

,Label,SMS,randy,prob,lived,responce,dha,garbage,pm,sem,...,workage,grow,acl03530150pm,woohoo,drunken,pose,nimya,dancin,26th,checking
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculer d'abord les constantes

Nous avons maintenant terminé le nettoyage de l'ensemble de formation et nous pouvons commencer à créer le filtre anti-spam. L'algorithme Naive Bayes devra répondre à ces deux questions de probabilité pour pouvoir classer les nouveaux messages:
$$ P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) $$$$ P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham) $$
De plus, pour calculer P (w$_i$|Spam) et P(w$_i$|Ham) dans les formules ci-dessus, nous devrons utiliser ces équations:
$$ P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} $$$$ P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}} $$

Certains des termes des quatre équations ci-dessus auront la même valeur pour chaque nouveau message. Nous pouvons calculer la valeur de ces termes une fois et éviter de refaire les calculs lorsqu'un nouveau message arrive. Ci-dessous, nous utiliserons notre ensemble d'entraînement pour calculer:

 -   P(Spam) et P(Ham)
 -   N$_{Spam}$, N$_{Ham}$, N$_{Vocabulary}$

Nous utiliserons également < Laplace smoothing > et définirons $\alpha = 1$.

In [15]:
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

p_spam = len(spam_messages)/len(training_set_clean)
p_ham = len(ham_messages)/len(training_set_clean)

n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_words_per_ham_message = ham_messages['SMS'].apply(len)

n_words_spam = n_words_per_spam_message.sum()
n_words_ham = n_words_per_ham_message.sum()
   
n_word_voca = len(word_counts_per_sms)

alpha = 1

Calcul des paramètres

Maintenant que nous avons les termes constants calculés ci-dessus, nous pouvons passer au calcul des paramètres $ P (w_i | Spam) $ et $ P (w_i | Ham) $. Chaque paramètre sera donc une valeur de probabilité conditionnelle associée à chaque mot du vocabulaire.

Les paramètres sont calculés à l'aide des formules:
$$ P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} $$$$ P(w_i|Ham) = 
\frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}} $$

In [16]:
dict_ham = {unique_word: 0 for unique_word in vocabulary}
dict_spam = {unique_word: 0 for unique_word in vocabulary}

In [17]:
spam_df = training_set_clean[training_set_clean['Label'] == 'spam']
ham_df = training_set_clean[training_set_clean['Label'] == 'ham']

In [18]:
for word in dict_ham:
    nwi_ham = ham_df[word].sum()
    p_wi_ham = (nwi_ham + alpha) / (n_words_ham + alpha * n_word_voca)
    dict_ham[word] = p_wi_ham
    
for word in dict_spam:
    nwi_spam = spam_df[word].sum()
    p_wi_spam = (nwi_spam + alpha) / (n_words_spam + alpha * n_word_voca)
    dict_spam[word] = p_wi_spam

## Classer un nouveau message

Maintenant que nous avons tous nos paramètres calculés, nous pouvons commencer à créer le filtre anti-spam. Le filtre anti-spam peut être compris comme une fonction qui:

  -   Prend en entrée un nouveau message (w$_1$, w$_2$, ..., w$_n$).
  -   Calcule P (Spam | w$_1$, w$_2$, ..., w$_n$) et P (Ham | w$_1$, w$_2$, ..., w$_n$).
  -   Compare les valeurs de P (Spam | w$_1$, w$_2$, ..., w$_n$) et P (Ham | w$_1$, w$_2$, ..., w$_n$), et:
      -   Si P (Ham | w$_1$, w$_2$, ..., w$_n$)> P (Spam | w$_1$, w$_2$, ..., w$_n$), alors le message est classé comme ham.
      -   Si P (Ham | w$_1$, w$_2$, ..., w$_n$) <P (Spam | w$_1$, w$_2$, ..., w$_n$), alors le message est classé comme spam.
      -   Si P (Ham | w$_1$, w$_2$, ..., w$_n$) = P (Spam | w$_1$, w$_2$, ..., w$_n$), alors l'algorithme peut demander une aide humaine.

In [19]:
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam 
    p_ham_given_message = p_ham 
    
    for word in message:
        if word in dict_spam:
            p_spam_given_message *= dict_spam[word]
        if word in dict_ham:
            p_ham_given_message *= dict_ham[word]
        

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [20]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [21]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Mesurer la précision du filtre anti-spam

Les deux résultats ci-dessus semblent prometteurs, mais voyons comment le filtre fonctionne bien sur notre ensemble de test, qui contient 1114 messages.

Nous allons commencer par écrire une fonction qui renvoie les classifications de Label au lieu de les imprimer.

In [22]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam 
    p_ham_given_message = p_ham 
    
    for word in message:
        if word in dict_spam:
            p_spam_given_message *= dict_spam[word]
        if word in dict_ham:
            p_ham_given_message *= dict_ham[word]
        
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [23]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head(5)

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [24]:
correct = 0
total = len(test_set)

for row in test_set.iterrows():
    if row[1]['Label'] == row[1]['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', round(correct/total*100,2),'%')

Correct: 1100
Incorrect: 14
Accuracy: 98.74 %


La précision est proche de 98,74%, ce qui est vraiment bon. Notre filtre anti-spam a examiné 1 114 messages qu'il n'a pas vus en formation et en a classé 1 100 correctement.